# Laboratorio 2

Modelación y Simulación

Edwin Ortega 22305 - Esteban Zambrano 22119 - Juan Pablo Solis 22102

Ejercicios en donde se necesita usar Julia

In [1]:
using JuMP
using HiGHS
using Ipopt
using Optimization